In [11]:
import torch
import random
import copy
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import optimizer
from torchvision import datasets, transforms
from matplotlib import pyplot as plt


In [12]:
MNIST = datasets.MNIST(root=".",
                       train=True,
                       transform = transforms.ToTensor(),
                       download=False)

MNIST_VAL = datasets.MNIST(root=".",
                           train=False,
                           transform = transforms.ToTensor(),
                           download=False)                    

In [13]:
dado, rotulo = MNIST[0]
print(dado.size())
print(MNIST.data.shape)

torch.Size([1, 28, 28])
torch.Size([60000, 28, 28])


In [14]:
dataloader = DataLoader(MNIST, batch_size=64)
dataloader_val = DataLoader(MNIST_VAL, batch_size=64)

In [15]:
model = nn.Sequential(
    nn.Conv2d(1, 20, 5),
    nn.MaxPool2d(2, stride=2),
    nn.Conv2d(20, 50, 5),
    nn.MaxPool2d(2, stride=2),
    nn.Flatten(),
    nn.Linear(800, 500),
    nn.ReLU(),
    nn.Linear(500, 10),
    nn.Softmax(dim=1)
)

In [16]:
def achatar(X):
    batch_size = X.size(0)
    flattened_X = X.view(batch_size, -1)
    return flattened_X

def permutar(images):
    permuted_images = images.clone()
    for i in range(images.size(0)):
        image = images[i]
        permuted_indices = torch.randperm(image.numel())
        permuted_image = image.view(-1)[permuted_indices].view(image.size())
        permuted_images[i] = permuted_image
    return permuted_images

In [17]:
learningRate = 0.001
optimizer = torch.optim.Adam(model.parameters(),lr=learningRate)
loss_fn = nn.CrossEntropyLoss()
epochs = 10

for e in range(epochs):
    #LOOP DE TREINAMENTO
    for i, (X, y) in enumerate(dataloader):
        X = permutar(X)
        yhat = model(X)
        loss = loss_fn(yhat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossvalue, pos = loss.item(), i * len(X)
        if i % 100 == 0:
            print(f"Epoch({e}:{pos}): [Loss: {lossvalue:.2f}]")
    #LOOP DE TESTE
    tloss = 0.0
    correct = 0.0
    size = len(dataloader_val.dataset)
    with torch.no_grad():
        for X, y in dataloader_val:
            pred = model(X)
            tloss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        print(f"Test error: \n Accuracy {100 * (correct/size):2.2f}%")
        

Epoch(0:0): [Loss: 2.30]
Epoch(0:6400): [Loss: 2.21]
Epoch(0:12800): [Loss: 2.29]
Epoch(0:19200): [Loss: 2.31]
Epoch(0:25600): [Loss: 2.23]
Epoch(0:32000): [Loss: 2.29]
Epoch(0:38400): [Loss: 2.07]
Epoch(0:44800): [Loss: 2.25]
Epoch(0:51200): [Loss: 2.26]
Epoch(0:57600): [Loss: 2.10]
Test error: 
 Accuracy 9.87%
Epoch(1:0): [Loss: 2.16]
Epoch(1:6400): [Loss: 2.15]
Epoch(1:12800): [Loss: 2.22]
Epoch(1:19200): [Loss: 2.31]
Epoch(1:25600): [Loss: 2.19]
Epoch(1:32000): [Loss: 2.26]
Epoch(1:38400): [Loss: 2.14]
Epoch(1:44800): [Loss: 2.24]
Epoch(1:51200): [Loss: 2.27]
Epoch(1:57600): [Loss: 2.13]
Test error: 
 Accuracy 10.09%
Epoch(2:0): [Loss: 2.17]
Epoch(2:6400): [Loss: 2.18]
Epoch(2:12800): [Loss: 2.24]
Epoch(2:19200): [Loss: 2.28]
Epoch(2:25600): [Loss: 2.21]
Epoch(2:32000): [Loss: 2.28]
Epoch(2:38400): [Loss: 2.11]
Epoch(2:44800): [Loss: 2.26]
Epoch(2:51200): [Loss: 2.26]
Epoch(2:57600): [Loss: 2.07]
Test error: 
 Accuracy 9.90%
Epoch(3:0): [Loss: 2.20]
Epoch(3:6400): [Loss: 2.16]
Epoc